In [ ]:
exp1 = "100-200*300-500+20"
exp2 = "50*6-3*2"
clas = {'+', '-', '*'}

# 문제 이해하기
# 목표 : 수식에 포함된 연산자의 우선순위를 자유롭게 재정의해 만들 수 있는 가장 큰 수 만들기
# 조건 (1) 연산자는 세 가지 +, -, *
# 조건 (2) 하나의 식에 같은 우선순위의 연산자는 있을 수 없음 (n!) 동일한 연산자가 식에 두 번 이상 출현할 수 있음
# 조건 (3) 수식의 길이는 3 이상 100 이하
# 조건 (4) 수식은 적어도 하나의 연산자를 포함하고 있음
# 조건 (5) 수식은 중위표기법으로 표현됨

# 추상화 하기
# 목표 : 가장 큰 계산값을 가지는 수식의 계산 순서를 정의하기

# 구체화 하기
# (안1) 식에서 피연산자와 연산자를 추출하여 각각의 리스트에 담기
# (안2) 식에서 피연산자-연산자-피연산자 쌍들을 모두 추출하여 리스트에 담기
# (안3) 후위연산식으로 바꿔 큐에 넣고, 우선순위에 따라 계산하는 방식

# 안3 . 후위연산식으로 변경
# (1) 순열 조합(?) 을 생성할 수 있어야 함 - 연산자 우선순위 만들기
# (2) 후위연산용 stack 구성


def convert_to_list(expression):
    import re
    converted = (re.split(r'([*+\-/])', expression.replace(' ', ''))) # 정규표현식
    number_list = []
    operator_list = []
    for comp in converted:
        try:
            number_list.append(int(comp))
        except:
            operator_list.append(comp)
    return number_list, operator_list
        

def make_priority_list(expression_list):
    return list(set([x for x in expression_list if x in ['+', '-', '*']]))

def make_permutations(origin_list, num):
    import copy
    result = []
    def permute(temp, remain_list):
        if len(temp) == num:
            result.append(temp[:]) # 이렇게 접근하는 게 깊은복사보다 더 빠름
        for i, comp in enumerate(remain_list):
            picked_comp = remain_list[i]
            permute(temp + [picked_comp], remain_list[:i] + remain_list[i+1:])
    permute([], origin_list)
    return result

def calc(comp1, comp2, operator):
    comp1 = int(comp1)
    comp2 = int(comp2)
    if operator == '+':
        return comp1 + comp2
    elif operator == '-':
        return comp1 - comp2
    elif operator == '*':
        return comp1 * comp2

def calc_expression(number_list, operator_list, priority_operator_set):
    for priority_operator in priority_operator_set:
        temp_number_list = []
        temp_operator_list = []
        for i, operator in enumerate(operator_list):
            if i == 0:
                if priority_operator == operator:
                    addition = calc(number_list[i], number_list[i+1], operator)
                    temp_number_list.append(addition)
                else:
                    temp_number_list.extend([number_list[i], number_list[i+1]])
                    temp_operator_list.append(operator)
            else:
                if priority_operator == operator:
                    addition = calc(temp_number_list.pop(), number_list[i+1], operator)
                    temp_number_list.append(addition)
                else:
                    temp_number_list.extend([number_list[i+1]])
                    temp_operator_list.append(operator)
        number_list = temp_number_list
        operator_list = temp_operator_list
        if len(number_list) == 1:
            return number_list[0]
    
def solution(expression):
    answer = 0
    number_list, operator_list = convert_to_list(expression)
    priority_list = make_priority_list(operator_list)
    operator_permutations = make_permutations(priority_list, len(priority_list))
    for operator_set in operator_permutations:
        result = abs(calc_expression(number_list, operator_list, operator_set))
        answer = result if result > answer else answer
    return answer

# 공부할 것
# (1) 정규표현식
# (2) stack 자료구조 만들기
# (3) 순열 만들기
# (4) 순열과 조합의 차이점
# (5) eval 함수
# 재귀함수에서는 return 자신함수() 이렇게 해야 (혹은 내부에 재귀) 반환된다고 함 아니면 반환값이 없음

# 주의할 것
# 문제 잘 읽기 절대값인지
# 리스트 초기화 temp_list = [] 꼭
# 잘 실수하는 부분 : number_list = temp_number_list.extend(number_list[i+2]) 이러면 Nonetype 이다.
# 순열이고 뭐고 내가 접근하는 방식이 효율적이고 좋은것인지 검토 필요


In [32]:
expression = exp1
result = solution(expression)
result

60420

In [40]:
# 다른사람 풀이
# 이걸 보고, 기본에 충실한 걸 보여주는 것보다 눈에 띄게 잘하고, 어떻게든 풀어내는 게 중요하다는 걸 깨달음
# 순열이고 뭐고 내가 접근하는 방식이 효율적이고 좋은것인지 검토 필요
def solution(expression):
    operations = [('+', '-', '*'),('+', '*', '-'),('-', '+', '*'),('-', '*', '+'),('*', '+', '-'),('*', '-', '+')]
    answer = []
    for op in operations:
        a = op[0]
        b = op[1]
        temp_list = []
        for e in expression.split(a):
            temp = [f"({i})" for i in e.split(b)]
            temp_list.append(f'({b.join(temp)})')
            print(temp_list)
        answer.append(abs(eval(a.join(temp_list)))) # eval
    return max(answer)

In [41]:
expression = exp1
result = solution(expression)
result

['((100)-(200*300)-(500))']
['((100)-(200*300)-(500))', '((20))']
['((100-200)*(300-500))']
['((100-200)*(300-500))', '((20))']
['((100))']
['((100))', '((200*300))']
['((100))', '((200*300))', '((500)+(20))']
['((100))']
['((100))', '((200)*(300))']
['((100))', '((200)*(300))', '((500+20))']
['((100-200))']
['((100-200))', '((300-500)+(20))']
['((100)-(200))']
['((100)-(200))', '((300)-(500+20))']


60420

2